In [1]:
import numpy as np
import pandas as pd
import yahoo_fin.stock_info as si
import datetime
import sqlite3 as sq

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
start = datetime.datetime(2023,8,22)
end = datetime.datetime(2023,8,23)
historical_datas = {}

In [3]:
tickers_csv = pd.read_excel('..\src\Copy of NYSE_Stocks.xlsx')
tickers_list= tickers_csv['Symbol'].tolist()

ticker_count = len(tickers_list)
print(f"Number of unique tickers: {ticker_count}")

Number of unique tickers: 8970


In [4]:
for idx, symbol in enumerate(tickers_list, start=1):  # start=1 makes idx start from 1 instead of 0
    try:
        historical_datas[symbol] = si.get_data(symbol, start_date=start, end_date=end, index_as_date=True)
        print(f"Fetching data for {symbol}: {idx}/{ticker_count}")
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        continue

Fetching data for A: 1/8970
Fetching data for AA: 2/8970
Fetching data for AAA: 3/8970
Fetching data for AAAU: 4/8970
Fetching data for AAC: 5/8970
Fetching data for AACG: 6/8970
Fetching data for AACI: 7/8970
Fetching data for AACT: 8/8970
Fetching data for AADI: 9/8970
Fetching data for AADR: 10/8970
Fetching data for AAIC: 11/8970
Fetching data for AAL: 12/8970
Fetching data for AAMC: 13/8970
Fetching data for AAME: 14/8970
Fetching data for AAN: 15/8970
Fetching data for AAOI: 16/8970
Fetching data for AAON: 17/8970
Fetching data for AAP: 18/8970
Fetching data for AAPB: 19/8970
Fetching data for AAPD: 20/8970
Fetching data for AAPL: 21/8970
Fetching data for AAPU: 22/8970
Fetching data for AAT: 23/8970
Fetching data for AAXJ: 24/8970
Fetching data for AB: 25/8970
Fetching data for ABBV: 26/8970
Fetching data for ABC: 27/8970
Fetching data for ABCB: 28/8970
Fetching data for ABCL: 29/8970
Fetching data for ABCM: 30/8970
Fetching data for ABEO: 31/8970
Fetching data for ABEQ: 32/8970

In [5]:
# Concatenate the dataframes in the dictionary
all_data = pd.concat(historical_datas.values())
all_data = all_data.reset_index().rename(columns={'index': 'date'})
cols = ['date', 'ticker'] + [col for col in all_data.columns if col not in ['date', 'ticker']]
all_data = all_data[cols]
# Check for duplicate 'ticker' columns and keep only one
if all_data.columns.duplicated().sum() > 0:
    all_data = all_data.loc[:, ~all_data.columns.duplicated()]

all_data['date'] = all_data['date'].astype(str)
all_data

,date,ticker,open,high,low,close,adjclose,volume
0,2023-08-22,A,117.849998,118.610001,117.330002,117.639999,117.639999,1422300
1,2023-08-22,A,117.849998,118.610001,117.330002,117.639999,117.639999,1422269
2,2023-08-22,AA,28.910000,29.090000,28.180000,28.780001,28.780001,4594000
3,2023-08-22,AA,28.910000,29.090000,28.180000,28.780001,28.780001,4430203
4,2023-08-22,AAA,24.799999,24.850000,24.799999,24.830000,24.830000,1900
...,...,...,...,...,...,...,...,...
17737,2023-08-22,ZYME,7.680000,7.935000,7.658000,7.860000,7.860000,393437
17738,2023-08-22,ZYNE,1.280000,1.300000,1.270000,1.300000,1.300000,497300
17739,2023-08-22,ZYNE,1.280000,1.300000,1.270000,1.300000,1.300000,496953
17740,2023-08-22,ZYXI,7.910000,8.100000,7.780000,7.980000,7.980000,283500


In [6]:
conn = sq.connect('../market_data.db')

try:
    df_db = pd.read_sql_query("SELECT * from price_action", conn)
except:
    df_db = pd.DataFrame(columns=['date', 'ticker', 'open', 'high', 'low', 'close', 'adjclose', 'volume'])

combined_data = pd.concat([df_db, all_data]).drop_duplicates(subset=['date', 'ticker'], keep='last')
combined_data.to_sql('price_action', conn, if_exists='replace', index=False)

conn.close()